# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    text = set(text)
    vocab_to_int = {entry: i for (i, entry) in enumerate(text)}
    #int_to_vocab = {i: entry for (entry, i) in enumerate(text)}
    int_to_vocab = dict(enumerate(text))
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_dict = {}
    token_dict['.'] = '||Period||'
    token_dict[','] = '||Comma||'
    token_dict['"'] = '||Quotation_Mark||'
    token_dict[';'] = '||Semicolon||'
    token_dict['!'] = '||Exclamation_Mark||'
    token_dict['?'] = '||Question_Mark||'
    token_dict['('] = '||Left_Parentheses||'
    token_dict[')'] = '||Right_Parentheses||'
    token_dict['--'] = '||Dash||'
    token_dict['\n'] = '||Return||'
    
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    Input = tf.placeholder(tf.int32, [None, None], name="input")
    Targets = tf.placeholder(tf.int32, [None, None], name="targets")
    LearningRate = tf.placeholder(tf.int32, None, name="learning_rate")
    
    return Input, Targets, LearningRate

    # I used this forum post to help resolve this error:
    # AssertionError: Learning Rate has wrong rank. Rank 2 found
    # https://discussions.udacity.com/t/get-inputs-types-and-shapes/327570/7

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
# keep probability global
keep_prob = 0.7

# LSTM size global
lstm_size = 2

def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size, state_is_tuple=True)
    cell = tf.contrib.rnn.MultiRNNCell([cell] * lstm_size)
    
    cell_state = cell.zero_state(batch_size, tf.float32)
    initialize_state = tf.identity(cell_state, name='initial_state')
    
    return(cell, initialize_state)

    # I used this discussion to solve my layer size issue
    # https://discussions.udacity.com/t/get-init-cell-issue/331648/10


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    word_embeddings = tf.Variable(tf.random_uniform((vocab_size, embed_dim)))
    embedded_word_ids = tf.nn.embedding_lookup(word_embeddings,input_data)

    return embedded_word_ids

    # sources used:
    # https://www.tensorflow.org/programmers_guide/embedding
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    
    return outputs, final_state

    # https://discussions.udacity.com/t/build-rnn-inputs-types/352538/8

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    embedded = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embedded)
    
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    return logits, final_state

    # sources:
    # https://discussions.udacity.com/t/assertionerror-final-state-wrong-shape-found-shape-2-2-128-256/405325
    # https://discussions.udacity.com/t/build-rnn-inputs-types/352538/8

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])
    
    ydata[-1] = xdata[0]
    # source: https://discussions.udacity.com/t/the-last-target-of-the-last-batch-should-be-the-first-input-of-the-first-batch-found-4476-4477-4478-4479-4480-but-expected-4476-4477-4478-4479-0/322359/2

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

    return np.array(list(zip(x_batches, y_batches)))

    # get_batches code taken from embeddings exercise

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 25
# RNN Size
rnn_size = 4
# Embedding Dimension Size
embed_dim = 4
# Sequence Length
seq_length = 10
# Learning Rate
learning_rate = 0.1
# Show stats for every n number of batches
show_every_n_batches = 5

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/276   train_loss = 8.822
Epoch   0 Batch    5/276   train_loss = 8.822
Epoch   0 Batch   10/276   train_loss = 8.822
Epoch   0 Batch   15/276   train_loss = 8.821
Epoch   0 Batch   20/276   train_loss = 8.821
Epoch   0 Batch   25/276   train_loss = 8.822
Epoch   0 Batch   30/276   train_loss = 8.822
Epoch   0 Batch   35/276   train_loss = 8.822
Epoch   0 Batch   40/276   train_loss = 8.821
Epoch   0 Batch   45/276   train_loss = 8.822
Epoch   0 Batch   50/276   train_loss = 8.821
Epoch   0 Batch   55/276   train_loss = 8.822
Epoch   0 Batch   60/276   train_loss = 8.822
Epoch   0 Batch   65/276   train_loss = 8.822
Epoch   0 Batch   70/276   train_loss = 8.822
Epoch   0 Batch   75/276   train_loss = 8.822
Epoch   0 Batch   80/276   train_loss = 8.821
Epoch   0 Batch   85/276   train_loss = 8.821
Epoch   0 Batch   90/276   train_loss = 8.822
Epoch   0 Batch   95/276   train_loss = 8.822
Epoch   0 Batch  100/276   train_loss = 8.821
Epoch   0 Batch  105/276   train_l

Epoch   3 Batch   72/276   train_loss = 8.822
Epoch   3 Batch   77/276   train_loss = 8.822
Epoch   3 Batch   82/276   train_loss = 8.821
Epoch   3 Batch   87/276   train_loss = 8.822
Epoch   3 Batch   92/276   train_loss = 8.821
Epoch   3 Batch   97/276   train_loss = 8.822
Epoch   3 Batch  102/276   train_loss = 8.821
Epoch   3 Batch  107/276   train_loss = 8.822
Epoch   3 Batch  112/276   train_loss = 8.821
Epoch   3 Batch  117/276   train_loss = 8.822
Epoch   3 Batch  122/276   train_loss = 8.822
Epoch   3 Batch  127/276   train_loss = 8.822
Epoch   3 Batch  132/276   train_loss = 8.821
Epoch   3 Batch  137/276   train_loss = 8.821
Epoch   3 Batch  142/276   train_loss = 8.821
Epoch   3 Batch  147/276   train_loss = 8.821
Epoch   3 Batch  152/276   train_loss = 8.821
Epoch   3 Batch  157/276   train_loss = 8.822
Epoch   3 Batch  162/276   train_loss = 8.822
Epoch   3 Batch  167/276   train_loss = 8.822
Epoch   3 Batch  172/276   train_loss = 8.822
Epoch   3 Batch  177/276   train_l

Epoch   6 Batch  154/276   train_loss = 8.822
Epoch   6 Batch  159/276   train_loss = 8.821
Epoch   6 Batch  164/276   train_loss = 8.822
Epoch   6 Batch  169/276   train_loss = 8.822
Epoch   6 Batch  174/276   train_loss = 8.822
Epoch   6 Batch  179/276   train_loss = 8.821
Epoch   6 Batch  184/276   train_loss = 8.822
Epoch   6 Batch  189/276   train_loss = 8.822
Epoch   6 Batch  194/276   train_loss = 8.822
Epoch   6 Batch  199/276   train_loss = 8.821
Epoch   6 Batch  204/276   train_loss = 8.821
Epoch   6 Batch  209/276   train_loss = 8.822
Epoch   6 Batch  214/276   train_loss = 8.822
Epoch   6 Batch  219/276   train_loss = 8.822
Epoch   6 Batch  224/276   train_loss = 8.821
Epoch   6 Batch  229/276   train_loss = 8.822
Epoch   6 Batch  234/276   train_loss = 8.821
Epoch   6 Batch  239/276   train_loss = 8.821
Epoch   6 Batch  244/276   train_loss = 8.821
Epoch   6 Batch  249/276   train_loss = 8.822
Epoch   6 Batch  254/276   train_loss = 8.821
Epoch   6 Batch  259/276   train_l

Epoch   9 Batch  221/276   train_loss = 8.821
Epoch   9 Batch  226/276   train_loss = 8.821
Epoch   9 Batch  231/276   train_loss = 8.821
Epoch   9 Batch  236/276   train_loss = 8.822
Epoch   9 Batch  241/276   train_loss = 8.822
Epoch   9 Batch  246/276   train_loss = 8.822
Epoch   9 Batch  251/276   train_loss = 8.822
Epoch   9 Batch  256/276   train_loss = 8.822
Epoch   9 Batch  261/276   train_loss = 8.822
Epoch   9 Batch  266/276   train_loss = 8.822
Epoch   9 Batch  271/276   train_loss = 8.821
Epoch  10 Batch    0/276   train_loss = 8.822
Epoch  10 Batch    5/276   train_loss = 8.822
Epoch  10 Batch   10/276   train_loss = 8.822
Epoch  10 Batch   15/276   train_loss = 8.821
Epoch  10 Batch   20/276   train_loss = 8.821
Epoch  10 Batch   25/276   train_loss = 8.822
Epoch  10 Batch   30/276   train_loss = 8.822
Epoch  10 Batch   35/276   train_loss = 8.822
Epoch  10 Batch   40/276   train_loss = 8.821
Epoch  10 Batch   45/276   train_loss = 8.822
Epoch  10 Batch   50/276   train_l

Epoch  13 Batch   17/276   train_loss = 8.822
Epoch  13 Batch   22/276   train_loss = 8.821
Epoch  13 Batch   27/276   train_loss = 8.821
Epoch  13 Batch   32/276   train_loss = 8.821
Epoch  13 Batch   37/276   train_loss = 8.821
Epoch  13 Batch   42/276   train_loss = 8.821
Epoch  13 Batch   47/276   train_loss = 8.822
Epoch  13 Batch   52/276   train_loss = 8.822
Epoch  13 Batch   57/276   train_loss = 8.822
Epoch  13 Batch   62/276   train_loss = 8.822
Epoch  13 Batch   67/276   train_loss = 8.822
Epoch  13 Batch   72/276   train_loss = 8.822
Epoch  13 Batch   77/276   train_loss = 8.822
Epoch  13 Batch   82/276   train_loss = 8.821
Epoch  13 Batch   87/276   train_loss = 8.822
Epoch  13 Batch   92/276   train_loss = 8.821
Epoch  13 Batch   97/276   train_loss = 8.822
Epoch  13 Batch  102/276   train_loss = 8.821
Epoch  13 Batch  107/276   train_loss = 8.822
Epoch  13 Batch  112/276   train_loss = 8.821
Epoch  13 Batch  117/276   train_loss = 8.822
Epoch  13 Batch  122/276   train_l

...

Epoch  35 Batch  230/276   train_loss = 8.822
Epoch  35 Batch  235/276   train_loss = 8.821
Epoch  35 Batch  240/276   train_loss = 8.821
Epoch  35 Batch  245/276   train_loss = 8.821
Epoch  35 Batch  250/276   train_loss = 8.822
Epoch  35 Batch  255/276   train_loss = 8.822
Epoch  35 Batch  260/276   train_loss = 8.822
Epoch  35 Batch  265/276   train_loss = 8.821
Epoch  35 Batch  270/276   train_loss = 8.821
Epoch  35 Batch  275/276   train_loss = 8.822
Epoch  36 Batch    4/276   train_loss = 8.821
Epoch  36 Batch    9/276   train_loss = 8.822
Epoch  36 Batch   14/276   train_loss = 8.822
Epoch  36 Batch   19/276   train_loss = 8.822
Epoch  36 Batch   24/276   train_loss = 8.822
Epoch  36 Batch   29/276   train_loss = 8.821
Epoch  36 Batch   34/276   train_loss = 8.822
Epoch  36 Batch   39/276   train_loss = 8.821
Epoch  36 Batch   44/276   train_loss = 8.822
Epoch  36 Batch   49/276   train_loss = 8.821
Epoch  36 Batch   54/276   train_loss = 8.821
Epoch  36 Batch   59/276   train_l

Epoch  39 Batch   21/276   train_loss = 8.821
Epoch  39 Batch   26/276   train_loss = 8.821
Epoch  39 Batch   31/276   train_loss = 8.822
Epoch  39 Batch   36/276   train_loss = 8.822
Epoch  39 Batch   41/276   train_loss = 8.821
Epoch  39 Batch   46/276   train_loss = 8.821
Epoch  39 Batch   51/276   train_loss = 8.822
Epoch  39 Batch   56/276   train_loss = 8.822
Epoch  39 Batch   61/276   train_loss = 8.821
Epoch  39 Batch   66/276   train_loss = 8.821
Epoch  39 Batch   71/276   train_loss = 8.822
Epoch  39 Batch   76/276   train_loss = 8.822
Epoch  39 Batch   81/276   train_loss = 8.821
Epoch  39 Batch   86/276   train_loss = 8.821
Epoch  39 Batch   91/276   train_loss = 8.822
Epoch  39 Batch   96/276   train_loss = 8.821
Epoch  39 Batch  101/276   train_loss = 8.822
Epoch  39 Batch  106/276   train_loss = 8.822
Epoch  39 Batch  111/276   train_loss = 8.822
Epoch  39 Batch  116/276   train_loss = 8.822
Epoch  39 Batch  121/276   train_loss = 8.822
Epoch  39 Batch  126/276   train_l

Epoch  42 Batch   93/276   train_loss = 8.821
Epoch  42 Batch   98/276   train_loss = 8.822
Epoch  42 Batch  103/276   train_loss = 8.821
Epoch  42 Batch  108/276   train_loss = 8.821
Epoch  42 Batch  113/276   train_loss = 8.821
Epoch  42 Batch  118/276   train_loss = 8.822
Epoch  42 Batch  123/276   train_loss = 8.822
Epoch  42 Batch  128/276   train_loss = 8.822
Epoch  42 Batch  133/276   train_loss = 8.822
Epoch  42 Batch  138/276   train_loss = 8.821
Epoch  42 Batch  143/276   train_loss = 8.822
Epoch  42 Batch  148/276   train_loss = 8.822
Epoch  42 Batch  153/276   train_loss = 8.822
Epoch  42 Batch  158/276   train_loss = 8.821
Epoch  42 Batch  163/276   train_loss = 8.822
Epoch  42 Batch  168/276   train_loss = 8.821
Epoch  42 Batch  173/276   train_loss = 8.821
Epoch  42 Batch  178/276   train_loss = 8.821
Epoch  42 Batch  183/276   train_loss = 8.821
Epoch  42 Batch  188/276   train_loss = 8.821
Epoch  42 Batch  193/276   train_loss = 8.822
Epoch  42 Batch  198/276   train_l

Epoch  45 Batch  160/276   train_loss = 8.822
Epoch  45 Batch  165/276   train_loss = 8.822
Epoch  45 Batch  170/276   train_loss = 8.821
Epoch  45 Batch  175/276   train_loss = 8.822
Epoch  45 Batch  180/276   train_loss = 8.821
Epoch  45 Batch  185/276   train_loss = 8.821
Epoch  45 Batch  190/276   train_loss = 8.821
Epoch  45 Batch  195/276   train_loss = 8.821
Epoch  45 Batch  200/276   train_loss = 8.822
Epoch  45 Batch  205/276   train_loss = 8.822
Epoch  45 Batch  210/276   train_loss = 8.822
Epoch  45 Batch  215/276   train_loss = 8.821
Epoch  45 Batch  220/276   train_loss = 8.822
Epoch  45 Batch  225/276   train_loss = 8.821
Epoch  45 Batch  230/276   train_loss = 8.822
Epoch  45 Batch  235/276   train_loss = 8.821
Epoch  45 Batch  240/276   train_loss = 8.821
Epoch  45 Batch  245/276   train_loss = 8.821
Epoch  45 Batch  250/276   train_loss = 8.822
Epoch  45 Batch  255/276   train_loss = 8.822
Epoch  45 Batch  260/276   train_loss = 8.822
Epoch  45 Batch  265/276   train_l

Epoch  48 Batch  232/276   train_loss = 8.821
Epoch  48 Batch  237/276   train_loss = 8.821
Epoch  48 Batch  242/276   train_loss = 8.822
Epoch  48 Batch  247/276   train_loss = 8.821
Epoch  48 Batch  252/276   train_loss = 8.822
Epoch  48 Batch  257/276   train_loss = 8.822
Epoch  48 Batch  262/276   train_loss = 8.822
Epoch  48 Batch  267/276   train_loss = 8.822
Epoch  48 Batch  272/276   train_loss = 8.822
Epoch  49 Batch    1/276   train_loss = 8.821
Epoch  49 Batch    6/276   train_loss = 8.822
Epoch  49 Batch   11/276   train_loss = 8.822
Epoch  49 Batch   16/276   train_loss = 8.822
Epoch  49 Batch   21/276   train_loss = 8.821
Epoch  49 Batch   26/276   train_loss = 8.821
Epoch  49 Batch   31/276   train_loss = 8.822
Epoch  49 Batch   36/276   train_loss = 8.822
Epoch  49 Batch   41/276   train_loss = 8.821
Epoch  49 Batch   46/276   train_loss = 8.821
Epoch  49 Batch   51/276   train_loss = 8.822
Epoch  49 Batch   56/276   train_loss = 8.822
Epoch  49 Batch   61/276   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [ ]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    return None, None, None, None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [ ]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    return None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [ ]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.